In [1]:
# General libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# For preprocessing
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from skimage.transform import resize
from skimage.color import rgb2lab, lab2rgb, rgb2gray
# For NN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, UpSampling2D
from tensorflow.keras.callbacks import EarlyStopping
# For output image
from skimage.io import imsave

%matplotlib inline

In [ ]:
# Get images
train_input_files = []
train_input_dir = "/Users/cleonwong/Desktop/Code/Colouring-FashionMNIST/Images/Train"

for (dirpath, dirnames, filenames) in os.walk(train_input_dir):
    train_input_files.extend(filenames)
    break
    
train_input = []
for filename in train_input_files
    
